In [4]:
import sympy as sp
from graphs import GraphDomain, GraphXAxis


gd = GraphDomain(-50, 50)

x = GraphXAxis(gd, 0.05)

x.limits

(-55.0, 55.0)

In [5]:
from enum import Enum

class TickMarkType(Enum):
    STANDARD = 1
    DEGREES = 2




In [9]:
from sympy import symbols, sqrt
x, y = symbols('x y')
expr = x + 2*y + sqrt(8)
expr

x + 2*y + 2*sqrt(2)